# ТОП 20 продаж SKU в категории 

In [ ]:
import pandas as pd

# Читаем Excel-файл
df = pd.read_excel('Top_20_in_group.xlsx')

# Переименуем столбцы для удобства (если названия отличаются, подставьте свои)
df.columns = ['Классификатор_ур1', 'Классификатор_ур2', 'Код_номенклатуры', 'Товар', 'Сумма_продаж']

# Группируем данные по "Классификатор_ур1" и сортируем внутри каждой группы по продажам
# Также берем топ-20 уникальных товаров (по "Код_номенклатуры" или "Товар", так как они уникальны)
top_20_by_group = (df.groupby('Классификатор_ур1')
                   .apply(lambda x: x.sort_values(by='Сумма_продаж', ascending=False)
                   .drop_duplicates(subset=['Код_номенклатуры'])  # Убираем дубликаты по уникальному коду
                   .head(20))  # Берем топ-20
                   .reset_index(drop=True))

# Сохраняем результат в новый Excel-файл
top_20_by_group.to_excel('Top_20_by_Classifier1.xlsx', index=False)

print("Топ-20 товаров по продажам для каждой категории сохранены в 'Top_20_by_Classifier1.xlsx'")

# Сглаживание Пиков

In [ ]:
import pandas as pd          # простоге скользящее среднее (SMA, Simple Moving Average)
import matplotlib.pyplot as plt

# === 1. Загрузка данных ===
file_path = 'SKU for prophet.xlsx'
df = pd.read_excel(file_path, sheet_name='Data')

# === 2. Сглаживание по каждому SKU ===
df = df.sort_values(['Код товара', 'Дата'])  # Сортировка по SKU и дате
df['Продажи_сглажено'] = df.groupby('Код товара')['Продажи, кг'].transform(
    lambda x: x.rolling(window=7, min_periods=1).mean()
)

# === 3. График для конкретного SKU ===
specific_sku = '187909'  # Указываем код как строку
# Пробуем преобразовать коды в строковый тип для единообразия
df['Код товара'] = df['Код товара'].astype(str).str.strip()

sku_df = df[df['Код товара'] == specific_sku]

if not sku_df.empty:
    print(f"Найдены данные для SKU {specific_sku}: {len(sku_df)} строк")
    plt.figure(figsize=(16, 8))
    plt.plot(sku_df['Дата'], sku_df['Продажи, кг'], label='Оригинальные данные', alpha=0.5)
    plt.plot(sku_df['Дата'], sku_df['Продажи_сглажено'], label='Сглаженные (SMA 7)', linewidth=2)
    plt.title(f'Сглаживание продаж для SKU {specific_sku}')
    plt.xlabel('Дата')
    plt.ylabel('Продажи, кг')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print(f"Код товара {specific_sku} не найден в данных после преобразования.")

# === 4. Сохранение в Excel ===
# output_path = 'SKU_smooth_output.xlsx'
# df.to_excel(output_path, index=False)
# print(f"Файл сохранён как: {output_path}")

In [ ]:
import pandas as pd        #  Скользящее медианное сглаживание
import matplotlib.pyplot as plt

# Загрузка данных
file_path = 'SKU for prophet.xlsx'
df = pd.read_excel(file_path, sheet_name='Data')

# Сортировка
df = df.sort_values(['Код товара', 'Дата'])
df['Код товара'] = df['Код товара'].astype(str).str.strip()

# Скользящая медиана
df['Продажи_сглажено'] = df.groupby('Код товара')['Продажи, кг'].transform(
    lambda x: x.rolling(window=7, min_periods=1).median()
)

# График для конкретного SKU
specific_sku = '187909'
sku_df = df[df['Код товара'] == specific_sku]

if not sku_df.empty:
    plt.figure(figsize=(16, 8))
    plt.plot(sku_df['Дата'], sku_df['Продажи, кг'], label='Оригинальные данные', alpha=0.5)
    plt.plot(sku_df['Дата'], sku_df['Продажи_сглажено'], label='Сглаженные (Медиана 7)', linewidth=2)
    plt.title(f'Сглаживание медианой для SKU {specific_sku}')
    plt.xlabel('Дата')
    plt.ylabel('Продажи, кг')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print(f"Код товара {specific_sku} не найден.")

In [ ]:
import pandas as pd            #Экспоненциальное сглаживание (EMA)
import matplotlib.pyplot as plt

# Загрузка данных
file_path = 'SKU for prophet.xlsx'
df = pd.read_excel(file_path, sheet_name='Data')

# Сортировка
df = df.sort_values(['Код товара', 'Дата'])
df['Код товара'] = df['Код товара'].astype(str).str.strip()

# Экспоненциальное сглаживание
df['Продажи_сглажено'] = df.groupby('Код товара')['Продажи, кг'].transform(
    lambda x: x.ewm(span=7, adjust=False).mean()
)

# График для конкретного SKU
specific_sku = '187909'
sku_df = df[df['Код товара'] == specific_sku]

if not sku_df.empty:
    plt.figure(figsize=(16, 8))
    plt.plot(sku_df['Дата'], sku_df['Продажи, кг'], label='Оригинальные данные', alpha=0.5)
    plt.plot(sku_df['Дата'], sku_df['Продажи_сглажено'], label='Сглаженные (EMA 7)', linewidth=2)
    plt.title(f'Экспоненциальное сглаживание для SKU {specific_sku}')
    plt.xlabel('Дата')
    plt.ylabel('Продажи, кг')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print(f"Код товара {specific_sku} не найден.")

In [ ]:
import pandas as pd                 #Savitzky-Golay
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

# Загрузка данных
file_path = 'SKU for prophet.xlsx'
df = pd.read_excel(file_path, sheet_name='Data')

# Сортировка
df = df.sort_values(['Код товара', 'Дата'])
df['Код товара'] = df['Код товара'].astype(str).str.strip()

# Применение Savitzky-Golay фильтра
df['Продажи_сглажено'] = df.groupby('Код товара')['Продажи, кг'].transform(
    lambda x: savgol_filter(x, window_length=7, polyorder=2) if len(x) >= 7 else x
)

# График для конкретного SKU
specific_sku = '187909'
sku_df = df[df['Код товара'] == specific_sku]

if not sku_df.empty:
    plt.figure(figsize=(16, 8))
    plt.plot(sku_df['Дата'], sku_df['Продажи, кг'], label='Оригинальные данные', alpha=0.5)
    plt.plot(sku_df['Дата'], sku_df['Продажи_сглажено'], label='Сглаженные (Savitzky-Golay)', linewidth=2)
    plt.title(f'Сглаживание Savitzky-Golay для SKU {specific_sku}')
    plt.xlabel('Дата')
    plt.ylabel('Продажи, кг')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print(f"Код товара {specific_sku} не найден.")

# Схлопывает магазины и строки в одну


In [ ]:
import pandas as pd

# Чтение данных из Excel-файла
df = pd.read_excel('1.xlsx')

# Объединяем столбцы "Вид деятельности субарендатора" и "Сумма арендной платы, руб" в одну строку
df['Вид деятельности субарендатора-Сумма арендной платы, руб'] = df['Вид деятельности субарендатора'] + '-' + df['Сумма арендной платы, руб'].astype(str)

# Группируем по магазину и собираем уникальные комбинации видов деятельности и сумм
result = df.groupby('Магазин')['Вид деятельности субарендатора-Сумма арендной платы, руб'].unique().reset_index()

# Преобразуем списки в строки, разделённые запятыми
result['Вид деятельности субарендатора-Сумма арендной платы, руб'] = result['Вид деятельности субарендатора-Сумма арендной платы, руб'].apply(lambda x: ', '.join(x))

# Сохраняем результат в новый Excel-файл
result.to_excel('result_table.xlsx', index=False)

# Выводим результат для проверки
print(result)

# Быстрый Исследовательский анализ(EDA) и визуализация

In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport

# Загрузка данных
df = pd.read_excel('test ydata.xlsx')

# Создание отчёта
profile = ProfileReport(df, title="Excel Data Report", explorative=True)

# Сохранение отчёта в HTML
profile.to_file("report.html")

# Автоматизация. Открытие.Обновление.Отправка

In [ ]:
import win32com.client
import os
import schedule
import time

# 1. Обновление сводных таблиц в Excel-файле
def update_pivot_tables(file_path):
    try:
        # Проверка существования файла
        if not os.path.exists(file_path):
            print(f"Файл {file_path} не найден.")
            return False
        
        # Инициализация Excel
        excel = win32com.client.Dispatch("Excel.Application")
        excel.Visible = False  # Скрыть Excel
        excel.DisplayAlerts = False  # Отключить предупреждения
        
        # Открытие файла
        workbook = excel.Workbooks.Open(file_path)
        
        # Обновление всех сводных таблиц на всех листах
        for worksheet in workbook.Worksheets:
            for pivot_table in worksheet.PivotTables():
                pivot_table.RefreshTable()
        
        # Сохранение и закрытие файла
        workbook.Save()
        workbook.Close()
        excel.Quit()
        
        print(f"Excel-файл {file_path} успешно обновлен (сводные таблицы).")
        return True
    except Exception as e:
        print(f"Ошибка при обновлении сводных таблиц: {e}")
        return False
    finally:
        # Очистка объектов
        workbook = None
        excel = None

# 2. Отправка email через Outlook
def send_email_outlook(recipients, cc, subject, body, attachment_path):
    try:
        # Инициализация Outlook
        outlook = win32com.client.Dispatch("Outlook.Application")
        
        # Создание нового письма
        mail = outlook.CreateItem(0)  # 0 = olMailItem
        
        # Настройка параметров письма
        mail.To = recipients
        mail.CC = cc
        mail.Subject = subject
        mail.Body = body
        
        # Добавление вложения
        if os.path.exists(attachment_path):
            mail.Attachments.Add(attachment_path)
        else:
            print(f"Вложение {attachment_path} не найдено.")
            return False
        
        # Отправка письма
        mail.Send()
        print("Письмо успешно отправлено через Outlook!")
        return True
    except Exception as e:
        print(f"Ошибка при отправке письма: {e}")
        return False
    finally:
        # Очистка объектов
        mail = None
        outlook = None

# 3. Основная задача
def job():
    # Параметры
    excel_file = r"C:/Users/bondarenkovv/Desktop/Python/алгоритмы ексель/test ydata.xlsx"
    recipients = "bondarenkovv@pudmarket.ru"
    cc = "bondarenkovv@pudmarket.ru"
    subject = "Отчет"
    body = "Добрый день! Отчёт во вложении."
    
    # Обновление сводных таблиц и отправка письма
    if update_pivot_tables(excel_file):
        send_email_outlook(recipients, cc, subject, body, excel_file)

# 4. Планирование задачи
schedule.every().day.at("09:00").do(job)

# 5. Запуск планировщика
def main():
    print("Планировщик запущен. Ожидание выполнения задачи в 09:00...")
    while True:
        schedule.run_pending()
        time.sleep(60)  # Проверка каждую минуту

if __name__ == '__main__':
    main()